<a href="https://colab.research.google.com/github/arteagac/xlogit/blob/master/examples/mixed_logit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the `xlogit` library

In [ ]:
!pip install xlogit

# Electricity Dataset

# Read data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/electricity_long.csv")

varnames = ["pf", "cl", "loc", "wk", "tod", "seas"]
X = df[varnames].values
y = df['choice'].values
alt =[1, 2, 3, 4]

## Fit the model

In [ ]:
from xlogit import MixedLogit

model = MixedLogit()
model.fit(X, y, 
          varnames, 
          alt=alt, 
          randvars={'pf': 'n','cl':'n','loc':'n','wk':'n','tod':'n','seas':'n'}, 
          mixby=df.id.values,
          n_draws = 600)
model.summary()

**** GPU Processing Enabled ****
Estimation succesfully completed after 50 iterations. Use .summary() to see the estimated values
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -0.9858550     0.0338870   -29.0924488      9.94e-96 ***
cl                      2.2874329     0.1218031    18.7797616      2.12e-54 ***
loc                    -9.5325339     0.3096535   -30.7845208      4.6e-102 ***
wk                     -9.6372933     0.2914006   -33.0723204     2.21e-110 ***
tod                     1.6544010     0.0856374    19.3186659      1.25e-56 ***
seas                   -0.2342388     0.0235791    -9.9341782      8.03e-20 ***
sd.pf                   0.2130129     0.0162241    13.1293917      1.65e-31 ***
sd.cl                  -1.7759333     0.1177779   -15.0786599      3.19e-39 ***
sd

# Fishing Dataset

## Read data

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/fishing_long.csv")

varnames = ['price','catch']
X = df[varnames].values
y = df['choice'].values

## Fit model

In [ ]:
from xlogit import MixedLogit

model = MixedLogit()
model.fit(X, y, varnames= varnames,
          alt=['beach', 'boat', 'charter', 'pier'],
          randvars = {'price': 'n', 'catch': 'n'})
model.summary()

**** GPU Processing Enabled ****
Estimation succesfully completed after 23 iterations. Use .summary() to see the estimated values
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
price                   1.3273109     0.1745541     7.6040067       4.3e-13 ***
catch                  -0.0273483     0.0022584   -12.1094448      7.24e-31 ***
sd.price               -1.5718667     0.3776818    -4.1618817      0.000146 ***
sd.catch               -0.0104654     0.0021118    -4.9556965      4.16e-06 ***
---------------------------------------------------------------------------
Significance:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Log-Likelihood= -1300.794


# Car Dataset

## Read data

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/arteagac/xlogit/master/examples/data/car100_long.csv")
df.price = -1*df.price/10000
df.operating_cost = -1*df.operating_cost

varnames = ['high_performance','medium_performance','price', 'operating_cost',
            'range', 'electric', 'hybrid'] 

X = df[varnames].values
y = df['choice'].values

## Fit the model

In [ ]:
from xlogit import MixedLogit

model = MixedLogit()
model.fit(X, y, varnames = varnames,
          alt=['car','bus','bike'],
          randvars = {'price': 'ln', 'operating_cost': 'n',
                      'range': 'ln', 'electric':'n', 'hybrid': 'n'}, 
          mixby = df.person_id.values, #Panel column
          n_draws = 100) 
model.summary()

**** GPU Processing Enabled ****
Estimation succesfully completed after 73 iterations. Use .summary() to see the estimated values
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
medium_performance      0.5838102     0.0933309     6.2552736      4.55e-08 ***
high_performance        0.1087706     0.1000104     1.0875922         0.439    
price                  -0.7065544     0.1088849    -6.4890017      1.56e-08 ***
operating_cost          0.0064776     0.0049081     1.3197622         0.333    
range                 -40.6821504 68213.1388143    -0.0005964         0.796    
electric               -0.3034144     0.1370607    -2.2137227        0.0711 .  
hybrid                  0.9509027     0.1432550     6.6378305      7.84e-09 ***
sd.price               -0.7961969     0.0758233   -10.5006880      3.99e-17 ***
sd